<a href="https://colab.research.google.com/github/muxywj/.github/blob/main/%EC%9D%BC%EB%AA%B0%EC%9D%BC%EC%B6%9C%20API%20%EC%97%B0%EB%8F%99%20%EC%88%98%EC%A0%95%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API 불러오기 테스트

In [9]:
import requests

serviceKey = 'Pm/kNFEx4ruhUTK4w5WFeS3weybFEum+p40tGIcQ6GkNQDV7M+SAaRTwqg5Xnt+CSdly/BCSoFl5wJgc86+gRg=='
locdate = '2026'
location = '세종'
dataType = 'xml'

url = 'https://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'
params = {'serviceKey' : serviceKey, 'locdate': locdate, 'location': location, 'dataType' : dataType, 'numOfRows': 365, 'pageNo': 1 }

response = requests.get(url, params=params)
print(f"Status Code: {response.status_code}")
print(response.text)

Status Code: 200
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><aste>1858  </aste><astm>0610  </astm><civile>1755  </civile><civilm>0714  </civilm><latitude>3629</latitude><latitudeNum>36.4833330</latitudeNum><location>세종</location><locdate>20260101</locdate><longitude>12717</longitude><longitudeNum>127.2833330</longitudeNum><moonrise>1451  </moonrise><moonset>0518  </moonset><moontransit>2239  </moontransit><naute>1827  </naute><nautm>0642  </nautm><sunrise>0743  </sunrise><sunset>1726  </sunset><suntransit>123415</suntransit></item><item><aste>1859  </aste><astm>0611  </astm><civile>1755  </civile><civilm>0714  </civilm><latitude>3629</latitude><latitudeNum>36.4833330</latitudeNum><location>세종</location><locdate>20260102</locdate><longitude>12717</longitude><longitudeNum>127.2833330</longitudeNum><moonrise>1553  </moonrise><moonset>0632  </moonset><moontransit>2346 

# 데이터 불러오기

## API 호출
2026년 1월 1일부터 12월 31일까지의 모든 날짜를 생성.


In [10]:
from datetime import date, timedelta

# Define start and end dates for 2026
start_date = date(2026, 1, 1)
end_date = date(2026, 12, 31)

# Initialize an empty list to store the dates
dates_2026 = []

# Loop through dates from start_date to end_date
current_date = start_date
while current_date <= end_date:
    # Format the date as 'YYYYMMDD' string and append to the list
    dates_2026.append(current_date.strftime('%Y%m%d'))
    # Move to the next day
    current_date += timedelta(days=1)

# Verify the first few and last few dates
print(f"First 5 dates: {dates_2026[:5]}")
print(f"Last 5 dates: {dates_2026[-5:]}")
print(f"Total number of dates: {len(dates_2026)}")

First 5 dates: ['20260101', '20260102', '20260103', '20260104', '20260105']
Last 5 dates: ['20261227', '20261228', '20261229', '20261230', '20261231']
Total number of dates: 365


## Collect Sunrise/Sunset Data

### Subtask:
생성된 각 날짜에 대해 한국천문연구원_출몰시각 정보 API(getAreaRiseSetInfo)를 호출하고, 응답 XML에서 지역의 일출(sunrise) 및 일몰(sunset) 시각을 추출


In [11]:
import xml.etree.ElementTree as ET

# Initialize an empty list to store the collected data
sunrise_sunset_data = []

# Parse the XML response obtained from the previous API call in F3X8OQZwCS_D
# The 'response' object should be available from the previous cell's execution.
root = ET.fromstring(response.text)

# Find all item elements containing the sunrise and sunset information
items = root.findall('.//item')

if items:
    for item in items:
        locdate_tag = item.find('locdate') # Extract locdate as well
        sunrise_tag = item.find('sunrise')
        sunset_tag = item.find('sunset')

        # Strip whitespace and handle None case for each tag
        current_locdate = locdate_tag.text.strip() if locdate_tag is not None else 'N/A_DATE'
        sunrise_time = sunrise_tag.text.strip() if sunrise_tag is not None else 'N/A_SUNRISE'
        sunset_time = sunset_tag.text.strip() if sunset_tag is not None else 'N/A_SUNSET'

        # Append the extracted data to the list
        sunrise_sunset_data.append({
            'date': current_locdate,
            'sunrise': sunrise_time,
            'sunset': sunset_time
        })
else:
    print("No item elements found in the XML response.")

# Print the first few entries to verify the collected data
print("Collected Sunrise/Sunset Data (first 5 entries):")
for entry in sunrise_sunset_data[:5]:
    print(entry)

print("Collected Sunrise/Sunset Data (last 5 entries):")
for entry in sunrise_sunset_data[-5:]:
    print(entry)

print(f"Total entries collected: {len(sunrise_sunset_data)}")

Collected Sunrise/Sunset Data (first 5 entries):
{'date': '20260101', 'sunrise': '0743', 'sunset': '1726'}
{'date': '20260102', 'sunrise': '0743', 'sunset': '1727'}
{'date': '20260103', 'sunrise': '0743', 'sunset': '1728'}
{'date': '20260104', 'sunrise': '0743', 'sunset': '1728'}
{'date': '20260105', 'sunrise': '0743', 'sunset': '1729'}
Collected Sunrise/Sunset Data (last 5 entries):
{'date': '20261227', 'sunrise': '0741', 'sunset': '1722'}
{'date': '20261228', 'sunrise': '0742', 'sunset': '1723'}
{'date': '20261229', 'sunrise': '0742', 'sunset': '1724'}
{'date': '20261230', 'sunrise': '0742', 'sunset': '1724'}
{'date': '20261231', 'sunrise': '0742', 'sunset': '1725'}
Total entries collected: 365


## Data 프레임 구성
수집된 날짜, 일출 시각, 일몰 시각 데이터를 포함하는 pandas DataFrame을 생성. DataFrame의 컬럼은 '날짜', '일출 시각', '일몰 시각'으로 구성


In [12]:
import pandas as pd

# Create a pandas DataFrame from the collected data
df_sunrise_sunset = pd.DataFrame(sunrise_sunset_data)

# Rename the columns to the specified Korean names
df_sunrise_sunset.columns = ['날짜', '일출 시각', '일몰 시각']

# Display the first 5 rows of the DataFrame to verify
print(df_sunrise_sunset.head())

         날짜 일출 시각 일몰 시각
0  20260101  0743  1726
1  20260102  0743  1727
2  20260103  0743  1728
3  20260104  0743  1728
4  20260105  0743  1729


##생성된 DataFrame을 엑셀로 저장

In [14]:
output_filename = 'sunrise_sunset_2026.xlsx'
df_sunrise_sunset.to_excel(output_filename, index=False)
print(f"DataFrame successfully saved to {output_filename}")

DataFrame successfully saved to sunrise_sunset_2026.xlsx


# 엑셀 다운로드

In [15]:
from google.colab import files

files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>